In [2]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf
import numpy as np

# tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
# print(tf.__version__)

# Model Generation Example
This notebook provides a simple example of how to generate a custom TFLite model.

In [ ]:
#------------------------------------#
#== Layer parameters ===
#------------------------------------#
M = 2
N = 2
K = 2

filters = 8
kernel_size = 2
in1 = 4
in2 = 4
in3 = 1
out1 = in1 + 1 - kernel_size
out2 = in2 + 1 - kernel_size
out3 = filters
cols = out1 * out2
depth = kernel_size * kernel_size * in3
dw_ks = out1
#------------------------------------#

#------------------------------------#
#== Model definition ===
#------------------------------------#

inputs = tf.keras.Input((in1, in2, in3))
inputs2 = tf.keras.Input((out1, out2, N))
x = tf.keras.layers.Conv2D(
    filters=filters,
    kernel_size=kernel_size,
    strides=1,
    padding="valid",
    activation="relu",
    use_bias=True,
    name="conv1",
)(inputs)
x = tf.keras.layers.Dense(N, use_bias=False)(x)
x = tf.keras.layers.Softmax(axis=3)(x)
x = tf.keras.layers.DepthwiseConv2D(
    kernel_size=(3, 3),
    strides=(1, 1),
    padding="same",
    activation="relu",
    use_bias=True,
    name="dwconv2",
)(x)
x = tf.keras.layers.Conv2DTranspose(
    filters=2,
    kernel_size=kernel_size,
    strides=(1, 1),
    padding="same",
    kernel_initializer="glorot_uniform",
    bias_initializer="zeros",
    use_bias=True,
)(x)
x = tf.keras.layers.Add()([x, x])
x = tf.keras.layers.ZeroPadding2D(padding=(1, 1))(x)
x = tf.keras.ops.mean(x, axis=[1, 2], keepdims=False)
#------------------------------------#

#------------------------------------#
#== Model Buid ===
#------------------------------------#

simple_model = tf.keras.Model(inputs=(inputs), outputs=(x))
simple_model.summary()
#------------------------------------#


# Fitting the model on random data and saving it as Keras model

In [ ]:
outdir = "./files/"  # change to your output directory
simple_model.compile(optimizer="sgd", loss="mse")
in_shape = [
    list(shape) if isinstance(shape, tuple) else shape
    for shape in simple_model.input_shape
]
out_shape = [
    list(shape) if isinstance(shape, tuple) else shape
    for shape in simple_model.output_shape
]
if isinstance(out_shape[0], list):
    out_shape = [[1 if dim is None else dim for dim in shape] for shape in out_shape]
else:
    out_shape = [1 if dim is None else dim for dim in out_shape]

if isinstance(in_shape[0], list):
    in_shape = [[1 if dim is None else dim for dim in shape] for shape in in_shape]
else:
    in_shape = [1 if dim is None else dim for dim in in_shape]


if isinstance(in_shape[0], list):
    x = [np.random.rand(*shape).astype(np.float32) for shape in in_shape]
else:
    x = np.random.rand(*in_shape).astype(np.float32)
if isinstance(out_shape[0], list):
    y = [np.random.rand(*shape).astype(np.float32) for shape in out_shape]
else:
    y = np.random.rand(*out_shape).astype(np.float32)
simple_model.fit(
    x,
    y,
    batch_size=1,
    epochs=2,
)

os.makedirs(outdir, exist_ok=True)
simple_model.save(os.path.join(outdir, "simple_model.keras"))

def representative_dataset():
    for _ in range(1):
        if isinstance(in_shape[0], list):
            print("Multiple Inputs")
            data = [np.random.rand(*shape).astype(np.float32) for shape in in_shape]
        else:
            print("Single Input")
            data = [np.random.rand(*in_shape).astype(np.float32)]

        yield data

# Convert and save the model to TFLite format


In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(simple_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT] # this enables INT8 quantization
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]  # this enables INT8 quantization for all the nodes
converter.inference_input_type = tf.int8 # Set this if you want give quantized input
converter.inference_output_type = tf.int8 # Set this if you want quantized output
tflite_model = converter.convert()
file_s = f"{outdir}/simple_model.tflite"
with open(file_s, "wb") as f:
    f.write(tflite_model)

print(file_s)